In [1]:
%run solution.ipynb

In [2]:
def calculate_delta_r1(T):
    return 2 * G0('Al', T) + 2 * G0('HCl', T) - 2 * G0('AlCl', T) - G0('H2', T)

def calculate_delta_r2(T):
    return G0('Al', T) + 2 * G0('HCl', T) - G0('AlCl2', T) - G0('H2', T)

def calculate_delta_r3(T):
    return 2 * G0('Al', T) + 6 * G0('HCl', T) - 2 * G0('AlCl3', T) - 3 * G0('H2', T)

In [3]:
Pa = 100000

def K1(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r1(T) / (R * T)) / Pa

def K2(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r2(T) / (R * T))

def K3(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r3(T) / (R * T)) / Pa



In [4]:
T = 623.15
print(K1(T))
print(K2(T))
print(K3(T))

7.497131271125371e-07
7.221267004644182e-08
4.062199476332757e-49


In [5]:
print(D('AlCl', T))
print(D('AlCl2', T))
print(D('AlCl3', T))
print(D('HCl', T))
print(D('H2', T))

0.000191129931554728
0.00014747988098603198
0.00015245155020045148
0.0002736394738546893
0.0008762807050502889


In [6]:
PG = {
   'AlCl':0,
   'AlCl2':0,
   'AlCl3':0,
   'H2':0,
   'HCl':10000,
   'N2':90000
}

# (D('AlCl', T) * (PG['AlCl'] - AlCl) + 2 * D('AlCl2', T) * (PG['AlCl2'] - AlCl2) + \
#         3 * D('AlCl3', T) * (PG['AlCl3'] - AlCl3))

T = 623.15
HCl, AlCl, H2, AlCl2, AlCl3 = sy.symbols('HCl AlCl H2 AlCl2 AlCl3')
X = sy.Matrix([
    K1(T) * AlCl**2 - K2(T) * AlCl2,
    ((D('AlCl', T) * (PG['AlCl'] - AlCl) + 2 * D('AlCl2', T) * (PG['AlCl2'] - AlCl2) + \
        3 * D('AlCl3', T) * (PG['AlCl3'] - AlCl3))/D('HCl', T) + PG['HCl'])**2 - K2(T) * AlCl2 * \
    ((D('AlCl', T) * (PG['AlCl'] - AlCl) + 2 * D('AlCl2', T) * (PG['AlCl2'] - AlCl2) + \
        3 * D('AlCl3', T) * (PG['AlCl3'] - AlCl3))/(2*D('H2', T)) + PG['H2']),
    (K2(T) * AlCl2)**3 - K3(T) * AlCl3**2
])
Y = [AlCl, AlCl2, AlCl3]
X.jacobian(Y)

Matrix([
[                                                                   1.49942625422507e-6*AlCl,                                                                      -7.22126700464418e-8,                                                                                          0],
[0.975730567746483*AlCl + 1.50578852394018*AlCl2 + 2.33482484539568*AlCl3 - 13969.4707684041, 1.50578852394018*AlCl + 2.3237962955969*AlCl2 + 3.60320007779103*AlCl3 - 21558.2757719156, 2.33482484539568*AlCl + 3.60320007779103*AlCl2 + 5.58700038604647*AlCl3 - 33427.5347162978],
[                                                                                          0,                                                             1.12969567064259e-21*AlCl2**2,                                                                -8.12439895266551e-49*AlCl3]])

In [7]:
ans = newton_method(X * (10 ** 50), Y, [1, 1, 1])
P = {'AlCl': ans[0], 'AlCl2': ans[1], 'AlCl3': ans[2]}
print(P)

8
{'AlCl': 0.0642136015641139, 'AlCl2': 0.333333333334369, 'AlCl3': 5982.84972479590}


In [8]:
def V(T, ro=2697):
    return (G('AlCl', T, PG, P) + G('AlCl2', T, PG, P) + G('AlCl3', T, PG, P)) * bank_td['Al']['mu'] * 1e9 / ro

In [9]:
print(G('AlCl', T, PG, P))
print(G('AlCl2', T, PG, P))
print(G('AlCl3', T, PG, P))
print(V(T))

-9.19895114691111e-9
-3.68463185938302e-8
-0.000683632212837233
-6839.69778313434
